In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import fastf1
from src.plotset import setup_plot, save_fig
from src.utils import compare_car_speeds
from fastf1 import plotting
import matplotlib as mpl
from matplotlib.collections import LineCollection

setup_plot()

In [ ]:
fastf1.Cache.enable_cache('./f1_cache')
fastf1.Cache.get_cache_info()

In [ ]:
session = fastf1.get_session(2025,15,'FP2')
session.load()

In [ ]:
drivers = session.drivers
drivers = [session.get_driver(drv).Abbreviation for drv in drivers]
drivers.remove('HAD')

In [ ]:
corner_inputs = {'low':[12],
                 'med':[8,13],
                 'high':[7,14]}
car_perf = compare_car_speeds(session=session,drivers=drivers,corner_inputs=corner_inputs)

In [ ]:
car_perf['color'] = car_perf.index.to_series().apply(lambda x: plotting.get_driver_color(identifier=x,session=session))

In [ ]:
# --- 1️⃣ Define ceilings (fastest driver speeds) ---
min_speed_ceiling = car_perf['LowSpeedAvg'].max()
max_speed_ceiling = car_perf['HighSpeedAvg'].max()

# --- 2️⃣ Calculate negative gaps (driver deficit) ---
car_perf['min_gap'] = car_perf['LowSpeedAvg'] - min_speed_ceiling  # ≤ 0
car_perf['max_gap'] = car_perf['HighSpeedAvg'] - max_speed_ceiling  # ≤ 0

# --- 3️⃣ Sort dataframe based on gaps (best performer first) ---
car_perf_sorted = car_perf.sort_values(by='min_gap', ascending=False)

# ----4️⃣ Sort again for max_gap (might differ from min_gap order) ----
car_perf_sorted2 = car_perf.sort_values(by='max_gap', ascending=False)

# Top driver in slow speeds
best_driver = car_perf_sorted.index[0]
best_speed = car_perf_sorted['LowSpeedAvg'].iloc[0]
best_color = car_perf_sorted['color'].iloc[0]

# Top driver in straights
best_driver2 = car_perf_sorted2.index[0]
best_speed2 = car_perf_sorted2['HighSpeedAvg'].iloc[0]
best_color2 = car_perf_sorted2['color'].iloc[0]

In [ ]:
# --- Plotting ---

setup_plot(xyticksize=20,axeslabel=24,figtitle=30)

fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# ---- Plot 1: Slow corner speed gaps ----
axes[0].bar(car_perf_sorted.index,
            car_perf_sorted['min_gap'],
            color=car_perf_sorted['color'])
axes[0].set_ylabel('Δ Speed (km/h)')
axes[0].set_title('Slow Speed Corners Deficit (compared to Best, 114 km/h)')
axes[0].axhline(0, color='black', linewidth=10)

# ---- Plot 2: Straight-line speed gaps ----
axes[1].bar(car_perf_sorted2.index,
            car_perf_sorted2['max_gap'],
            color=car_perf_sorted2['color'])
axes[1].set_ylabel('Δ Speed (km/h)')
axes[1].set_title('High Speed Corners Deficit (compared to Best, 268 km/h)')
axes[1].axhline(0, color='black', linewidth=10)

axes[0].set_xlim(-0.5, len(car_perf_sorted) - 0.5)
axes[1].set_xlim(-0.5, len(car_perf_sorted2) - 0.5)

axes[0].text(
    x=0, y=-2.5,                           # position (x = bar index, y = just below axhline)
    s=f"{best_speed:.0f}",           # text to display
    color=best_color, fontsize=24,
    ha='center', va='bottom', fontweight='bold'
)

axes[1].text(
    x=0, y=-1.1,                           # position (x = bar index, y = just below axhline)
    s=f"{best_speed2:.0f}",           # text to display
    color=best_color2, fontsize=24,
    ha='center', va='bottom', fontweight='bold'
)

plt.tight_layout()
plt.show()

In [ ]:
save_fig(fig=fig,name='speed_deficit',loc='Reel13')

In [ ]:
lap = session.laps.pick_fastest()

In [ ]:
x = lap.telemetry['X']
y = lap.telemetry['Y']
color = lap.telemetry['Speed']

In [ ]:
theta = np.deg2rad(0)  # Convert 45° to radians
x_rot = x * np.cos(theta) - y * np.sin(theta)
y_rot = x * np.sin(theta) + y * np.cos(theta)

In [ ]:
points = np.array([x_rot, y_rot]).T.reshape(-1,1,2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 11))
fig.set_facecolor('#000000')
ax.axis('off')
ax.plot(x_rot, y_rot,
        color='#000000', linestyle='-', linewidth=16, zorder=0)

norm = plt.Normalize(color.min(), color.max())
colormap = mpl.cm.plasma
lc = LineCollection(segments, cmap=colormap, norm=norm,
                    linestyle='-', linewidth=14)

lc.set_array(color)

line = ax.add_collection(lc)

cbaxes = fig.add_axes([0.5, 0.9, 0.5, 0.05])
normlegend = mpl.colors.Normalize(vmin=color.min(), vmax=color.max())
legend = mpl.colorbar.ColorbarBase(cbaxes, norm=normlegend, cmap=colormap,
                                   orientation="horizontal")

for label in legend.ax.get_xticklabels():
    label.set_fontsize(16)
    label.set_fontweight('bold')
    label.set_color('w')

legend.ax.set_title('Speed (kmph)',color='w',fontdict=dict(fontsize=18,fontweight='bold'))
plt.tight_layout()
plt.show()

In [ ]:
save_fig(fig=fig,name='speed_map',loc='Reel13')